In [23]:
import numpy as np
import seaborn as sns
import pandas as pd 

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.utils.data as data
import torch.nn.functional as F

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder



In [4]:
torch.cuda.is_available()

True

In [17]:
df = pd.read_excel("Demographic_List.xlsx")
X = np.load('samples_correlation.npy')

In [47]:
# Binary enconding 
Y = df["Condition"].to_numpy()

l_encoder = LabelEncoder()
i_encoded = l_encoder.fit_transform(Y)
print(i_encoded)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 0 0 2 2 2 2 0 0 0 0 2 2 2 2 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 2 2 0 0 0 0 2 2 2 2 0 0 0 0 0 0 2 2 0 0 2 2 0 0 0 0 2 2 2 2 0
 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 0 0 2 2 2 2 2 2 0 0 2 2 2 2 0 0 0 0 0 0 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 2 2 1 1 2 2 2 2 2 2 2 1 1 1 1 2 2
 1 1 2 2 1 1 1 1 2 2 2 2 2 2 2 2 2 2 1 1 1 1 2 2 2 2 1 1 1 1 2 2 1 1 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [51]:
# Onehot enconding 
o_encoder = OneHotEncoder(sparse=False)
i_encoded = i_encoded.reshape(len(i_encoded), 1)
o_encoded = o_encoder.fit_transform(i_encoded)
print(o_encoded)

Y = o_encoded

[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [52]:
X = Variable(torch.from_numpy(X), requires_grad=False)
Y = Variable(torch.from_numpy(Y), requires_grad=False)

In [53]:
Y

tensor([[0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        ...,
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.]], dtype=torch.float64)

In [62]:
from torchsummary import summary

In [123]:
# Network Initialization 

model = nn.Sequential(
          nn.Conv2d(1,16,3),
          nn.Conv2d(16,16,3),
          nn.MaxPool2d(2),
          nn.Dropout(),
          nn.Conv2d(16,32,3),
          nn.Conv2d(32,32,3),
          nn.MaxPool2d(3),
          nn.Dropout(),
          nn.Flatten(),
          nn.Linear(32, 160),
          nn.Dropout(),
          nn.Linear(160, 3)
        )
model.to("cuda")

summary(model, (1,20,20))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 18, 18]             160
            Conv2d-2           [-1, 16, 16, 16]           2,320
         MaxPool2d-3             [-1, 16, 8, 8]               0
           Dropout-4             [-1, 16, 8, 8]               0
            Conv2d-5             [-1, 32, 6, 6]           4,640
            Conv2d-6             [-1, 32, 4, 4]           9,248
         MaxPool2d-7             [-1, 32, 1, 1]               0
           Dropout-8             [-1, 32, 1, 1]               0
           Flatten-9                   [-1, 32]               0
           Linear-10                  [-1, 160]           5,280
          Dropout-11                  [-1, 160]               0
           Linear-12                    [-1, 3]             483
Total params: 22,131
Trainable params: 22,131
Non-trainable params: 0
---------------------------------